### The following code uses the Globus Python SDK to submit a data transfer from the NCAR GLADE endpoint to the NCAR Campaign Storage endpoint

For more information, see the tutorial provided on the Globus Python SDK documentation pages at https://globus-sdk-python.readthedocs.io/en/stable/tutorial/

In [1]:
from globus_sdk import *
import json

### Start the Globus Auth flow

In [2]:
CLIENT_ID = '<YOUR-CLIENT-ID>'

client = NativeAppAuthClient(CLIENT_ID)
client.oauth2_start_flow(refresh_tokens=True)

authorize_url = client.oauth2_get_authorize_url()
print('Please go to this URL and login: {0}'.format(authorize_url))

Please go to this URL and login: https://auth.globus.org/v2/oauth2/authorize?client_id=c0614ae8-2667-465f-9d50-67924be812f7&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=openid+profile+email+urn%3Aglobus%3Aauth%3Ascope%3Atransfer.api.globus.org%3Aall&state=_default&response_type=code&code_challenge=gx4-lcqrZeAosUSzp0uvsQrTN5Rmq3izPMsB6CKBrRU&code_challenge_method=S256&access_type=offline


### Get auth code, exchange code for tokens

In [3]:
# this is to work on Python2 and Python3 -- you can just use raw_input() or
# input() for your specific version
get_input = getattr(__builtins__, 'raw_input', input)
auth_code = get_input(
    'Please enter the code you get after login here: ').strip()
token_response = client.oauth2_exchange_code_for_tokens(auth_code)

Please enter the code you get after login here:  tZK1rXs1PHDNZUz7k6alcSJv3j1E9T


In [4]:
globus_auth_data = token_response.by_resource_server['auth.globus.org']
globus_transfer_data = token_response.by_resource_server['transfer.api.globus.org']

# most specifically, you want these tokens as strings
TRANSFER_TOKEN = globus_transfer_data['access_token']
TRANSFER_RT = globus_transfer_data['refresh_token']
TRANSFER_EXPIRES_AT = globus_transfer_data['expires_at_seconds']

AUTH_TOKEN = globus_auth_data['access_token']
AUTH_RT = globus_auth_data['refresh_token']
AUTH_EXPIRES_AT = globus_auth_data['expires_at_seconds']

### Print the Transfer data response

In [5]:
for a in globus_transfer_data:
    print (a,':',globus_transfer_data[a])

scope : urn:globus:auth:scope:transfer.api.globus.org:all
access_token : AgEvKjmrMwBlyzOKJmD72KxOG6Epqkbxk31o2B0nbdoaw4lxzniwC4DP5VXG3moaDJpxmEM57dPbo5Uyk9aXOtVmbX
refresh_token : Ag50lMy3XJJNQynY23emPeQeXn5yNpBbe9awl8Vx45W49zEy2Xs4UP1ddBKp4aMe2889qWdNO5OzkKyKndj7qkx6zGW5x
token_type : Bearer
expires_at_seconds : 1565886297
resource_server : transfer.api.globus.org


### Instantitate the transfer client using your refresh token

In [8]:
tc_authorizer = RefreshTokenAuthorizer(TRANSFER_RT, client, access_token=TRANSFER_TOKEN, 
                                       expires_at=TRANSFER_EXPIRES_AT)
tc = TransferClient(authorizer=tc_authorizer)

### Search for the NCAR GLADE and NCAR Campaign Store endpoints and get the endpoint IDs

In [11]:
source_endpoint = tc.endpoint_search('NCAR GLADE')[0]['id']
destination_endpoint = tc.endpoint_search('NCAR Campaign Storage')[0]['id']

### Create transfer data object containing the data to be transferred

In [25]:
transfer_data = TransferData(transfer_client=tc,
							 source_endpoint=source_endpoint,
							 destination_endpoint=destination_endpoint,
							 label='NCAR example transfer')

### Add files or directories to be transferred.  Note source_path is relative to the source endpoint base path.

In [ ]:
source_files = ['file1.txt', 'file2.txt', 'file3.txt']

source_path = '<DIRECTORY-PATH-ON-GLADE>'
dest_path = '<DIRECTORY-PATH-ON-NCAR-CAMPAIGN-STORAGE>'

for file in source_files:
    source_file = "{0}{1}".format(source_path, source_files[file])
    dest_file = "{0}{1}".format(dest_path, source_files[file])
    transfer_data.add_item(source_file, dest_file)

### Activate both the source and destination endpoints

In [27]:
tc.endpoint_autoactivate(source_endpoint)
tc.endpoint_autoactivate(destination_endpoint)

TransferResponse({u'code': u'AutoActivated.GlobusOnlineCredential', u'resource': u'/endpoint/dabdcd87-6d04-11e5-ba46-22000b92c6ec/autoactivate', u'DATA_TYPE': u'activation_result', u'expires_in': -1, u'length': 0, u'endpoint': u'tcram#cisl-toulon.scd.ucar.edu', u'request_id': u'qKwIdUBnt', u'expire_time': u'2018-08-31 20:04:58+00:00', u'message': u'Endpoint activated successfully using Globus Online credentials.', u'DATA': [], u'oauth_server': None, u'subject': u'/C=US/O=Globus Consortium/OU=Globus Connect User/CN=tcram'})

### Submit the transfer and print the task ID

In [28]:
task_id = tc.submit_transfer(transfer_data)
print "Task ID: {}".format(task_id['task_id'])